In [1]:
import os
import pandas as pd

In [3]:
# Répertoire de base où se trouve le notebook
base_dir = os.getcwd()

# Liste des annotateurs et versions
annotators = ["AA", "AB", "CF", "CKP", "DN", "LM", "MB"]
authors = ["CARRAUD", "DAUDET"]
versions = ["Kraken-base", "REF", "TesseractFra-PNG"]

# Dictionnaire pour stocker les données
annotations_data = {}

# Parcourir les annotateurs, auteurs et versions
for annotator in annotators:
    annotator_dir = os.path.join(base_dir, f"{annotator}_Annotations")
    if not os.path.exists(annotator_dir):
        print(f"Dossier manquant pour l'annotateur : {annotator}")
        continue

    for author in authors:
        # Le dossier peut être `AA_CARRAUD` ou juste `CARRAUD`
        author_dir = os.path.join(annotator_dir, f"{annotator}_{author}")
        if not os.path.exists(author_dir):
            author_dir = os.path.join(annotator_dir, author)  # Essayer sans le préfixe annotateur
            if not os.path.exists(author_dir):
                print(f"Dossier manquant pour : {annotator} {author}")
                continue

        for version in versions:
            # Rechercher tous les fichiers contenant les mots-clés
            file_name_pattern = f"_{author}_petit" if author == "DAUDET" else f"_{author}_petite"
            file_name_pattern += f"-Jeanne_{version}" if author == "CARRAUD" else f"-chose_{version}"
            
            # Recherche des fichiers correspondant
            files = [file for file in os.listdir(author_dir) if file_name_pattern in file and file.endswith(".csv")]
            
            if files:
                for file in files:
                    file_path = os.path.join(author_dir, file)
                    try:
                        # Chargement avec gestion des erreurs
                        data = pd.read_csv(file_path, on_bad_lines='skip')  # Ignorer les lignes problématiques
                        annotations_data.setdefault(author, {}).setdefault(version, []).append(data)
                    except Exception as e:
                        print(f"Erreur lors du chargement : {file_path}\n{e}")
            else:
                print(f"Fichier manquant pour : {annotator} {author} {version}")

# Vérification : Nombre de fichiers chargés par auteur et version
for author, versions_data in annotations_data.items():
    for version, files in versions_data.items():
        print(f"Auteur : {author}, Version : {version}, Fichiers chargés : {len(files)}")


Dossier manquant pour : AB DAUDET
Auteur : CARRAUD, Version : Kraken-base, Fichiers chargés : 7
Auteur : CARRAUD, Version : REF, Fichiers chargés : 7
Auteur : CARRAUD, Version : TesseractFra-PNG, Fichiers chargés : 7
Auteur : DAUDET, Version : Kraken-base, Fichiers chargés : 6
Auteur : DAUDET, Version : REF, Fichiers chargés : 6
Auteur : DAUDET, Version : TesseractFra-PNG, Fichiers chargés : 6


In [7]:
import os
import pandas as pd
from collections import Counter
from functools import partial

# Répertoire de base où se trouve le notebook
base_dir = os.getcwd()

# Liste des annotateurs, auteurs et versions
annotators = ["AA", "AB", "CF", "CKP", "DN", "LM", "MB"]
authors = ["CARRAUD", "DAUDET"]
versions = ["Kraken-base", "REF", "TesseractFra-PNG"]

# Fonction pour calculer la majorité
def get_majority(votes):
    count = Counter(votes)
    majority_vote = count.most_common(1)
    return majority_vote[0][0] if majority_vote else None

# Fonction pour calculer Label_Maj avec gestion de NA
def compute_label_maj(row, files):
    corrections = []
    for file in files:
        if "Correction" in file.columns:
            if row.name < len(file):  # Vérifier que l'index existe
                corrections.append(file.loc[row.name, "Correction"])
    
    # Compter les occurrences
    correction_count = Counter(corrections)
    if correction_count["NA"] > len(files) / 2:  # Si "NA" est majoritaire
        return None  # Pas d'annotation retenue
    elif pd.notna(row["Correction"]) and row["Correction"] != "NA":
        return row["Correction"]  # Annotation dans Correction
    elif pd.notna(row["Label"]):
        return row["Label"]  # Annotation spaCy
    else:
        return None  # Pas d'annotation

# Fonction pour calculer Quaero_Maj
def compute_quaero_maj(row, files):
    if row["Label_Maj"] == "LOC":
        votes = []
        for file in files:
            if "Quaero" in file.columns:
                # Utiliser l'index global pour accéder à la ligne correspondante
                if row.name < len(file):  # Vérifier que l'index existe dans le fichier
                    value = file.loc[row.name, "Quaero"]
                    if pd.notna(value):  # Ajouter uniquement les valeurs non nulles
                        votes.append(value)
        return get_majority(votes)
    else:
        return None

# Dictionnaire pour stocker les fichiers consolidés
annotations_data = {}
consolidated_data = {}

# Lecture des fichiers CSV avec le bon séparateur (;)
for annotator in annotators:
    annotator_dir = os.path.join(base_dir, f"{annotator}_Annotations")
    if not os.path.exists(annotator_dir):
        print(f"Dossier manquant pour l'annotateur : {annotator}")
        continue

    for author in authors:
        # Le dossier peut être `AA_CARRAUD` ou juste `CARRAUD`
        author_dir = os.path.join(annotator_dir, f"{annotator}_{author}")
        if not os.path.exists(author_dir):
            author_dir = os.path.join(annotator_dir, author)  # Essayer sans le préfixe annotateur
            if not os.path.exists(author_dir):
                print(f"Dossier manquant pour : {annotator} {author}")
                continue

        for version in versions:
            # Rechercher tous les fichiers contenant les mots-clés
            file_name_pattern = f"_{author}_petit" if author == "DAUDET" else f"_{author}_petite"
            file_name_pattern += f"-Jeanne_{version}" if author == "CARRAUD" else f"-chose_{version}"
            
            # Recherche des fichiers correspondant
            files = [file for file in os.listdir(author_dir) if file_name_pattern in file and file.endswith(".csv")]
            
            if files:
                for file in files:
                    file_path = os.path.join(author_dir, file)
                    try:
                        # Spécifier le séparateur lors de la lecture
                        data = pd.read_csv(file_path, sep=';', on_bad_lines='skip')  # Utilise ; comme séparateur
                        annotations_data.setdefault(author, {}).setdefault(version, []).append(data)
                    except Exception as e:
                        print(f"Erreur lors du chargement : {file_path}\n{e}")
            else:
                print(f"Fichier manquant pour : {annotator} {author} {version}")

# Consolidation des données
for author, versions_data in annotations_data.items():
    for version, files in versions_data.items():
        # Fusionner tous les fichiers pour cette version/auteur
        merged_df = pd.concat(files, ignore_index=True)
        
        # Vérifier et normaliser les colonnes nécessaires
        required_columns = ["Token", "Label", "Quaero", "Correction"]
        for col in required_columns:
            if col not in merged_df.columns:
                merged_df[col] = None  # Ajouter une colonne vide si absente
        
        # Appliquer la fonction pour Label_Maj et Quaero_Maj
        compute_label_maj_partial = partial(compute_label_maj, files=files)
        compute_quaero_maj_partial = partial(compute_quaero_maj, files=files)

        merged_df["Label_Maj"] = merged_df.apply(compute_label_maj_partial, axis=1)
        merged_df["Quaero_Maj"] = merged_df.apply(compute_quaero_maj_partial, axis=1)

        # Conserver uniquement les colonnes finales
        final_df = merged_df[["Token", "Label_Maj", "Quaero_Maj"]]
        
        # Ajouter au dictionnaire
        consolidated_data.setdefault(author, {})[version] = final_df

# Sauvegarder les fichiers consolidés
output_dir = os.path.join(base_dir, "Vote_Majoritaire")
os.makedirs(output_dir, exist_ok=True)

for author, versions_data in consolidated_data.items():
    for version, df in versions_data.items():
        output_file = f"{author}_{version}_consolidé.csv"
        output_path = os.path.join(output_dir, output_file)
        df.to_csv(output_path, index=False, encoding="utf-8")
        print(f"Fichier sauvegardé : {output_path}")


Dossier manquant pour : AB DAUDET
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\CARRAUD_Kraken-base_consolidé.csv
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\CARRAUD_REF_consolidé.csv
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\CARRAUD_TesseractFra-PNG_consolidé.csv
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\DAUDET_Kraken-base_consolidé.csv
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\DAUDET_REF_consolidé.csv
Fichier sauvegardé : C:\Users\user\OneDrive\Desktop\M1S2\Mémoire\2. Formatage données entraînement modèles\Vote_Majoritaire\DAUDET_TesseractFra-PNG_consolidé.csv
